In [19]:
# -*- coding: utf-8 -*-

# TRANSFERÊNCIA DE ESTILO NEURAL BASEADO NO EXEMPLO DA DOCUMENTAÇÃO DO KERAS
# VEJA O CÓDIGO ORIGINAL EM: https://keras.io/examples/neural_style_transfer/

from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import numpy as np
from scipy.optimize import fmin_l_bfgs_b

from keras.applications import vgg19
from keras import backend as K

import os

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFilter

base_image_path = '../data/style/wave.jpg'
style_reference_image_path = '../data/content/nalport.jpg'
result_prefix = '.'

# número máximo de iterações
iterations = 50

# melhora mínima do loss. 
# caso a diferença entre a iteração anterior e a atual seja menor que essa taxa,
# o treinamento será encerrado imediatamente
min_loss = 1000000

# pesos dos diferentes funções loss
total_variation_weight = 1.0
style_weight = 1.0
content_weight = 0.025

# taxa da camada alpha na mesclagem da imagem base e a imagem gerada
# valores mais baixos resultam em fotos com o estilo artístico mais forte
blend = 0.25

# o filtro Sharpen pode melhorar os detalhes da imagem gerada
use_sharpen = False

# dimensões da imagem gerada
width, height = load_img(base_image_path).size
img_ncols = 1600 
img_nrows = 900  

In [20]:
# função auxiliar para abrir, redimensionar e transformar as imagens em tensores
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# função auxiliar para converter um tensor em uma imagem válida
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remover zero-center pela média de pixels
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# obter representação de tensores das nossas imagens
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

# isso vai conter a nossa imagem gerada
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
else:
    combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

# combinar as 3 imagens em um único tensor do Keras
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

# buildar a rede VGG19 com as nossas 3 images como input
# o modelo será inicializado com os pesos pré-treinados do ImageNet
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# Obter os symbolic outputs de cada "camada chave" (nós demos nomes únicos a eles)
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# computar o loss do estilo neural:
# primeiro nós temos que definir 4 funções auxiliares

# gram matrix de um tensor de imagem (produto externo das features)
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# o style loss tem como objetivo manter o estilo da imagem referenciada na imagem gerada
# ele é baseado nas gram matrices (as quais captam o estilo)
# dos feature maps da imagem de estilo referenciada e da imagem gerada
    
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

# uma loss function auxiliar
# que serve para manter o conteúdo da imagem base na imagem gerada

def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# a terceira loss function, total variation loss,
# serve para manter a imagem gerada coerente
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


# combinar as 3 loss funcions em um único scalar
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

with tf.GradientTape() as tape:
    loss = K.variable(0.0)
    loss = loss + content_weight * content_loss(base_image, combination_image)
    loss = loss + style_weight * style_loss(style_reference_image, combination_image)
    loss = loss + total_variation_weight * total_variation_loss(combination_image)
# obter os grandientes da imagem relacionados ao loss
grads = tape.gradient(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)


def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

    
# a classe Evaluator possibilita calcular o loss
# e os gradientes de uma só vez, enquanto os obtemos
# através de duas funções separadas, "loss" e "grads".
# isso foi feito porque scipy.optimize requer funções separadas
# para loss e gradiente, mas calculá-los separadamente seria ineficiente

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


evaluator = Evaluator()


# rodar a otimização baseada no scipy (L-BFGS) nos pixels da imagem gerada
# a fim de minimizar o loss do estilo neural
x = preprocess_image(base_image_path)
last_loss, min_val, info = None, None, None

for i in range(iterations):
    print('Início da iteração ', i)
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Loss atual: ', min_val)

    if last_loss != None:
        dif = last_loss - min_val
        if dif < min_loss:
            print('Encerrando treinamento na iteração', i)            
            break
    last_loss = min_val

# aprimorar efeito artístico mesclando
# a imagem de conteúdo com a imagem gerada
            
img = deprocess_image(x.copy())
img = array_to_img(img)
img.save('generated.jpg')

base = Image.open(base_image_path)
base = base.resize((img_ncols, img_nrows))

img_final = Image.blend(img, base, alpha=blend)
if use_sharpen == True:
    img_final = img_final.filter(ImageFilter.SHARPEN)
img_final.save(result_prefix)

  <tf.Variable 'Variable:0' shape=(1, 900, 1600, 3) dtype=float32>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.
Model loaded.


AssertionError: 